In [14]:
!pip install -q pandas matplotlib seaborn numpy plotly.express
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
from shapely.geometry import Point

In [15]:
tech_salaries = pd.read_csv('/Users/annamezhlauk/Desktop/ECO225Project/Data/Tech_salaries.csv')
startups_main = pd.read_csv('/Users/annamezhlauk/Desktop/ECO225Project/Data/Startups/objects.csv', low_memory = False)

In [16]:
# dropping observations with "spam" position names
tech_salaries = tech_salaries.drop([44, 53, 267, 281, 522, 586, 646, 789, 1303, 1422, 1423, 1424, 1555, 1595, 1640, 1642], axis=0)


# dropping observations where compensation isn't in USD
tech_salaries = tech_salaries.drop([1448, 1132, 860, 766, 689, 677, 387, 21], axis=0)

# dropping improbable salaries that could be data entry mistakes
tech_salaries = tech_salaries[(tech_salaries['annual_base_pay'] >= 30000) & (tech_salaries['annual_base_pay'] <= 3000000
                                                                           )].reset_index(drop=True)

In [17]:
startups_main.rename(columns = {"normalized_name": "employer_name"}, inplace = True)
startups_main.rename(columns = {"entity_id": "startup_id"}, inplace = True)

In [18]:
startups_main['startup_id'] = startups_main['startup_id'].astype('Int64')
startups_and_salaries = tech_salaries.merge(startups_main, how="left", on="employer_name")

In [19]:
startups_and_salaries.head()

,index,salary_id,employer_name,location_name,location_state,location_country,location_latitude,location_longitude,job_title,job_title_category,...,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at
0,0,1,opower,"san francisco, ca",CA,US,37.77,-122.41,systems engineer,Engineering,...,2010-11-29,3.0,65700000.0,2012-02-15,2013-09-01,3.0,27.0,svetla,2009-08-13 14:06:00,2013-07-25 20:10:17
1,1,3,walmart,"bentonville, ar",AR,US,36.36,-94.20,senior developer,Software,...,NaN,0.0,0.0,2012-02-19,2013-11-29,3.0,85.0,NaN,2009-08-06 18:53:35,2013-10-30 21:48:04
2,2,4,vertical knowledge,"cleveland, oh",OH,US,41.47,-81.67,software engineer,Software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,6,netapp,waltham,NaN,NaN,NaN,NaN,mts,Other,...,NaN,0.0,0.0,2011-05-09,2012-10-15,3.0,46.0,initial-importer,2008-01-04 07:41:24,2009-09-24 06:26:32
4,4,12,apple,cupertino,NaN,NaN,NaN,NaN,software engineer,Software,...,NaN,0.0,0.0,2013-03-05,2013-12-11,4.0,469.0,initial-importer,2007-12-22 04:28:53,2013-12-04 06:16:19


In [ ]:
startups_and_salaries_clean = startups_and_salaries.dropna(subset=['location_latitude', 'location_longitude'])

In [ ]:
startups_and_salaries_clean = startups_and_salaries_clean.copy()
startups_and_salaries_clean.loc[:, "Coordinates"] = list(zip(
    startups_and_salaries_clean.location_longitude, 
    startups_and_salaries_clean.location_latitude))

In [ ]:
startups_and_salaries_clean.loc[:, "Coordinates"] = startups_and_salaries_clean["Coordinates"].apply(Point)

In [ ]:
s_and_s_geo = gpd.GeoDataFrame(startups_and_salaries_clean, geometry="Coordinates")

In [ ]:
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

In [ ]:
fig, gax = plt.subplots(figsize=(10, 10))

world.plot(ax=gax, edgecolor='black', color='white')

gax.set_xlabel('longitude')
gax.set_ylabel('latitude')

gax.spines['top'].set_visible(False)
gax.spines['right'].set_visible(False)


s_and_s_geo.plot(ax=gax, column='annual_base_pay', cmap='Greens', alpha=0.5, legend=True, scheme='quantiles',)

plt.show()